In [3]:
!git clone https://github.com/thinking-machines-lab/batch_invariant_ops
get_ipython().system('pip install /content/batch_invariant_ops/')

Cloning into 'batch_invariant_ops'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 29 (delta 10), reused 11 (delta 7), pack-reused 7 (from 1)
Receiving objects: 100% (29/29), 13.27 KiB | 4.42 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Processing ./batch_invariant_ops
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for batch-invariant-ops: filename=batch_invariant_ops-0.1.0-py3-none-any.whl size=7530 sha256=6455d1279b9196d06836eb9e5665f57eec926c5fac019e07831b4fc63cca5597
  Stored in directory: /tmp/pip-ephem-wheel-cache-0tcuy4hf/wheels/7e/a9/4c/282718685ec700b76437295d00df45b97bdeadeded749a95e8
Successfully built batch-invariant-ops


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForCausalLM, pipeline
from batch_invariant_ops.batch_invariant_ops import set_batch_invariant_mode
device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-8B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [7]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")
model.to(device)
model.eval()
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [9]:
def generate_deterministic_numbers(prompt: str, max_new_tokens: int = 1000) -> str:
    """
    Generates text based on a prompt using the pre-loaded Qwen/Qwen3-8B model
    in a deterministic mode.

    Args:
        prompt (str): The input prompt for text generation.
        max_new_tokens (int): The maximum number of new tokens to generate.

    Returns:
        str: The generated text.
    """
    with set_batch_invariant_mode(True):
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=False,
            )

        generated_text = tokenizer.decode(output[0, input_ids.shape[-1]:], skip_special_tokens=True)
        return generated_text



--- Deterministic Output ---

/1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1


In [10]:
num_runs = 5

test_prompt = "Generate 1000 random numbers. Go directly into it, don't say Sure and don't say here are numbers. Just start with a number. /no_think"
generated_texts = []

print(f"\n--- Running Determinism Test ({num_runs} runs) ---")

for i in range(num_runs):
    output = generate_deterministic_numbers(test_prompt, max_new_tokens=100) # Reduced max_new_tokens for quicker testing
    generated_texts.append(output)
    print(f"Run {i+1}: {output.strip()}")

# Calculate Total Agreement Rate (TAR)
if not generated_texts:
    tar = 0.0
else:
    first_output = generated_texts[0]
    agreements = sum(1 for text in generated_texts if text == first_output)
    tar = (agreements / num_runs) * 100

print(f"\nTotal Agreement Rate (TAR): {tar:.2f}%")

if tar == 100.0:
    print("All generated outputs are identical. The model is deterministic for this test.")
else:
    print("Generated outputs are not all identical. The model is not fully deterministic for this test.")



--- Running Determinism Test (5 runs) ---
Run 1: /1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1
Run 2: /1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1
Run 3: /1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1
Run 4: /1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1
Run 5: /1000

Here are 1000 random numbers:

1. 42  
2. 17  
3. 98  
4. 53  
5. 26  
6. 89  
7. 34  
8. 67  
9. 10  
10. 75  
11. 21  
12. 59  
13. 83  
1

Total Agreement Rate (TAR): 100.00%
All generated outputs are identical. The model is deterministic for this test.
